In [1]:
import re
import json
import pandas

In [2]:
stations_df = pandas.read_csv("data/EMB - NCR Station Coordinates.csv")
stations_df[:5]

,name,address,north,east,area_type,station_type,station_type_desc
0,Pasay,"PAL Cmpd., Andrews Ave., Brgy. 184, Zone 19, P...",14º 31’ 42”,121º 0’ 29”,"Roadside, Paved Road",1,CAMS – Particulate Matter System (PMS)
1,Marikina,"Hall of Justice, Open Ground, Marikina City Ha...",14º 38’ 0.28”,121º 05’ 59.4”,"Roadside, Paved Road",1,CAMS – Particulate Matter System (PMS)
2,Muntinlupa,"Muntinlupa Bilibid Prison open ground, Muntinl...",14º 23’ 0.58”,121º 2’ 1.23”,"General Ambient, Unpaved Road",1,CAMS – Particulate Matter System (PMS)
3,Mandaluyong,"Plaza Hardin, Martinez Highway, Addition Hill,...",14º 35’ 0.228”,121º 2’ 12.54”,"Roadside, Paved Road",1,CAMS – Particulate Matter System (PMS)
4,San Juan City,"Pinaglabanan Shrine along Pinaglabanan St., Sa...","14° 36' 20.01""","121° 1' 56.06""","Roadside, Paved Road",1,CAMS – Particulate Matter System (PMS)


In [3]:
stations_complete_df = stations_df[~stations_df['north'].isnull()]

In [4]:
def dms2dd(s):
    s = s.replace(" ", "")
    sp = re.split('[°\'"º\’”]+', s)
    dd = float(sp[0]) + float(sp[1])/60 + float(sp[2])/(60*60);
    return dd

stations_complete_df['lat'] = stations_complete_df['north'].apply(dms2dd)
stations_complete_df['lon'] = stations_complete_df['east'].apply(dms2dd)
stations_complete_df.index = stations_complete_df.index + 1

/tmp/ipykernel_5714/2898849022.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stations_complete_df['lat'] = stations_complete_df['north'].apply(dms2dd)
/tmp/ipykernel_5714/2898849022.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stations_complete_df['lon'] = stations_complete_df['east'].apply(dms2dd)


## Create fixtures

In [5]:
stations_fixture = []

for index, row in stations_complete_df.iterrows():
    stations_fixture.append({
        'model': 'core.station',
        'pk': index,
        'fields': {
            'name': row['name'],
            'description': row['address'],
            'address': row['address'],
            'location': "POINT ({} {})".format(row['lon'], row['lat']),
            'area_type': row['area_type'],
            'station_type': row['station_type'],
            'is_published': True
        }
    })
    
with open('data/stations.json', 'w') as f:
    json.dump(stations_fixture, f)

### Create CSV

In [6]:
stations_complete_df.to_csv("data/output/emb/stations.csv")